In [49]:
import requests
import json
import pandas as pd

### Anwendungsbeispiel 1: Automatisierte Übersetzung mit DeepL

Read the [docs](https://www.deepl.com/docs-api)

In [2]:
# Abruf Auth Token aus externer Datei
with open('token.json') as f:
    credentials = json.load(f)
    AUTH_KEY_DEEPL = credentials['AUTH_TOKEN_DEEPL']

# Manuelle Eingabe des Auth Token (nicht empfohlen!)
# AUTH_KEY_DEEPL = '<INSERT TOKEN>'

In [17]:
ENDPOINT = 'https://api-free.deepl.com/v2/translate'

text = """Data literacy is the ability to read, understand, create, and communicate data as information. Much like literacy as a general concept, data literacy focuses on the competencies involved in working with data.
It is, however, not similar to the ability to read text since it requires certain skills involving reading and understanding data."""

HEADER = {'Authorization': 'DeepL-Auth-Key ' + AUTH_KEY_DEEPL}

PARAMS = {'source_lang': 'EN',
          'target_lang': 'DE',
          'text': text}

response = requests.get(ENDPOINT, headers=HEADER, params=PARAMS)
response, HEADER

(<Response [200]>,
 {'Authorization': 'DeepL-Auth-Key a931b7ec-d78a-e654-7810-ca5686d66f33:fx'})

In [18]:
response

<Response [200]>

In [19]:
response.request.url

'https://api-free.deepl.com/v2/translate?source_lang=EN&target_lang=DE&text=Data+literacy+is+the+ability+to+read%2C+understand%2C+create%2C+and+communicate+data+as+information.+Much+like+literacy+as+a+general+concept%2C+data+literacy+focuses+on+the+competencies+involved+in+working+with+data.%0AIt+is%2C+however%2C+not+similar+to+the+ability+to+read+text+since+it+requires+certain+skills+involving+reading+and+understanding+data.'

In [20]:
response.json()

{'translations': [{'detected_source_language': 'EN',
   'text': 'Datenkompetenz ist die Fähigkeit, Daten als Informationen zu lesen, zu verstehen, zu erstellen und zu kommunizieren. Ähnlich wie die Lesekompetenz als allgemeines Konzept konzentriert sich die Datenkompetenz auf die Kompetenzen, die mit der Arbeit mit Daten verbunden sind.\nSie ist jedoch nicht vergleichbar mit der Fähigkeit, Texte zu lesen, da sie bestimmte Fähigkeiten zum Lesen und Verstehen von Daten erfordert.'}]}

In [ ]:
response.json()['translations'][0]['text']

In [21]:
def translate(text, source_lang, target_lang):
        
    ENDPOINT = 'https://api-free.deepl.com/v2/translate'
    
    with open('token.json') as f:
        credentials = json.load(f)
        AUTH_KEY_DEEPL = credentials['AUTH_TOKEN_DEEPL']

    # AUTH_KEY_DEEPL = '<INSERT TOKEN>'
    
    HEADER = {'Authorization': 'DeepL-Auth-Key ' + AUTH_KEY_DEEPL}

    PARAMS = {'source_lang': 'EN',
              'target_lang': 'DE',
              'text': text}
    
    response = requests.get(ENDPOINT, headers=HEADER, params=PARAMS).json()['translations'][0]['text']
    return response

In [22]:
sentences = ["Data literacy is the ability to read, understand, create, and communicate data as information.",
             "Much like literacy as a general concept, data literacy focuses on the competencies involved in working with data.",
             "It is, however, not similar to the ability to read text since it requires certain skills involving reading and understanding data."]

for sent in sentences:
    print(translate(sent, 'EN', 'DE'))

Datenkompetenz ist die Fähigkeit, Daten als Informationen zu lesen, zu verstehen, zu erstellen und zu vermitteln.
Ähnlich wie bei der Alphabetisierung als allgemeinem Konzept liegt der Schwerpunkt bei der Datenkompetenz auf den Kompetenzen, die mit der Arbeit mit Daten verbunden sind.
Sie ist jedoch nicht vergleichbar mit der Fähigkeit, Texte zu lesen, da sie bestimmte Fähigkeiten zum Lesen und Verstehen von Daten erfordert.


*Hinweis: Es gibt sogar ein separates Python package (`deepl`), was die Benutzung der API noch einfacher macht!*

### Anwendungsbeispiel 2: Transkription mit AssemblyAI

Read the [docs](https://docs.assemblyai.com/)

Schritt 1: Upload der Audio-Datei

In [23]:
UPLOAD_ENDPOINT = 'https://api.assemblyai.com/v2/upload'

with open('token.json') as f:
    credentials = json.load(f)
    AUTH_TOKEN_ASSEMBLYAI = credentials['AUTH_TOKEN_ASSEMBLYAI']

# AUTH_TOKEN_ASSEMBLYAI = '<INSERT TOKEN>'

HEADER = {'authorization': AUTH_TOKEN_ASSEMBLYAI,
          'content-type': 'application/json'}

with open('recording.m4a', 'rb') as f:
    audio_file = f.read()
    
response = requests.post(UPLOAD_ENDPOINT, headers=HEADER, data=audio_file)

In [24]:
response

<Response [200]>

In [25]:
response.json()

{'upload_url': 'https://cdn.assemblyai.com/upload/bee93736-9d72-4b3b-af14-70e097d0412d'}

Schritt 2: Starten der Transkription

Inzwischen werden auch Transkriptionen in [Spanisch, Französisch, Deutsch und Italienisch](https://docs.assemblyai.com/#supported-languages) unterstützt.

In [26]:
TRANSCRIPT_ENDPOINT = 'https://api.assemblyai.com/v2/transcript'

UPLOAD_URL = response.json()['upload_url']

JSON = {'audio_url': UPLOAD_URL,
        'language_code': 'en',
        'speaker_labels': 'true'}

reponse = requests.post(TRANSCRIPT_ENDPOINT, headers=HEADER, json=JSON)

In [27]:
response

<Response [200]>

In [28]:
reponse.json()

{'id': 'olc217e4jj-0fec-4348-b2a8-fda06e656511',
 'language_model': 'assemblyai_default',
 'acoustic_model': 'assemblyai_default',
 'language_code': 'en_us',
 'status': 'queued',
 'audio_url': 'https://cdn.assemblyai.com/upload/bee93736-9d72-4b3b-af14-70e097d0412d',
 'text': None,
 'words': None,
 'utterances': None,
 'confidence': None,
 'audio_duration': None,
 'punctuate': True,
 'format_text': True,
 'dual_channel': None,
 'webhook_url': None,
 'webhook_status_code': None,
 'speed_boost': False,
 'auto_highlights_result': None,
 'auto_highlights': False,
 'audio_start_from': None,
 'audio_end_at': None,
 'word_boost': [],
 'boost_param': None,
 'filter_profanity': False,
 'redact_pii': False,
 'redact_pii_audio': False,
 'redact_pii_audio_quality': None,
 'redact_pii_policies': None,
 'redact_pii_sub': None,
 'speaker_labels': True,
 'content_safety': False,
 'iab_categories': False,
 'content_safety_labels': {},
 'iab_categories_result': {},
 'language_detection': False,
 'custom_

Schritt 3: Download des Transkriptes

*Hinweis: Laut [Anbieter](https://docs.assemblyai.com/#processing-times) veranschlagt der Prozess etwa 30% der Länge der Audio-Datei.*

In [32]:
TRANSCRIPT_ID = reponse.json()['id']

response = requests.get(TRANSCRIPT_ENDPOINT + '/' + TRANSCRIPT_ID, headers=HEADER)
response.json()

{'id': 'olc217e4jj-0fec-4348-b2a8-fda06e656511',
 'language_model': 'assemblyai_default',
 'acoustic_model': 'assemblyai_default',
 'language_code': 'en_us',
 'status': 'completed',
 'audio_url': 'https://cdn.assemblyai.com/upload/bee93736-9d72-4b3b-af14-70e097d0412d',
 'text': 'Data literacy is the ability to meet, understand, create, and communicate data as information. Much like literacy. As a general concept, data literacy focuses on the competition involved in working with data. It is, however, not similar to the ability to read text since it requires certain skills involving reading and understanding. Data Background as data collection and data sharing become routine, data analysis and big data become common ideas in the news, business, government and society becomes more and more important for students, citizens and leaders to have some data literacy. The concept is associated with data science, which is concerned with data analysis, usually through automated means and the inter

In [33]:
response.json()['text']

'Data literacy is the ability to meet, understand, create, and communicate data as information. Much like literacy. As a general concept, data literacy focuses on the competition involved in working with data. It is, however, not similar to the ability to read text since it requires certain skills involving reading and understanding. Data Background as data collection and data sharing become routine, data analysis and big data become common ideas in the news, business, government and society becomes more and more important for students, citizens and leaders to have some data literacy. The concept is associated with data science, which is concerned with data analysis, usually through automated means and the interpretation and application of the results.'

### Anwendungsbeispiel 3: Musik-Empfehlungen mit Spotify

Read the [docs](https://developer.spotify.com/documentation/web-api/)

Schritt 1: Erstellen eines Authentication Token

In [36]:
AUTH_ENDPOINT = 'https://accounts.spotify.com/api/token'

with open('token.json') as f:
    credentials = json.load(f)
    CLIENT_ID_SPOTIFY = credentials['CLIENT_ID_SPOTIFY']
    CLIENT_SECRET_SPOTIFY = credentials['CLIENT_SECRET_SPOTIFY'] 

# CLIENT_ID_SPOTIFY = '<INSERT CLIENT_ID>'
# CLIENT_SECRET_SPOTIFY = '<INSERT CLIENT_SECRET>'

DATA = {'grant_type': 'client_credentials',
        'client_id': CLIENT_ID_SPOTIFY,
        'client_secret': CLIENT_SECRET_SPOTIFY}

response = requests.post(AUTH_ENDPOINT, data=DATA)

In [37]:
response

<Response [200]>

In [38]:
response.json()

{'access_token': 'BQD_lVC82Gt6_sliptI8ni2IpdbFDoMZshhQbgWCpqVYhqdO04RWb9-pI9KHzaNLG1kjad4fUPwHOc3MIiE',
 'token_type': 'Bearer',
 'expires_in': 3600}

Schritt 2: Abruf von Song Eigenschaften

*Hinweis: Die Spotify API bietet eine Fülle an verschiedenen Abfragemöglichkeiten (d.h. [Endpoints](https://developer.spotify.com/documentation/web-api/reference/#/)). Zudem gibt es ein separates Python package (`spotipy`), was die Benutzung der API noch einfacher macht!*

In [39]:
FEATURES_ENDPOINT = 'https://api.spotify.com/v1/audio-features/'

AUTH_TOKEN_SPOTIFY = response.json()['access_token']

TRACK_ID = '1bBglCIHeIwR78kJXJzkZo'

HEADER = {'Content-Type': 'application/json',
           'Authorization': 'Bearer ' + AUTH_TOKEN_SPOTIFY}

response = requests.get(FEATURES_ENDPOINT + TRACK_ID, headers=HEADER)

In [40]:
response

<Response [200]>

In [41]:
response.json()

{'danceability': 0.617,
 'energy': 0.807,
 'key': 0,
 'loudness': -9.83,
 'mode': 1,
 'speechiness': 0.0378,
 'acousticness': 0.0164,
 'instrumentalness': 0.695,
 'liveness': 0.0927,
 'valence': 0.124,
 'tempo': 125.004,
 'type': 'audio_features',
 'id': '1bBglCIHeIwR78kJXJzkZo',
 'uri': 'spotify:track:1bBglCIHeIwR78kJXJzkZo',
 'track_href': 'https://api.spotify.com/v1/tracks/1bBglCIHeIwR78kJXJzkZo',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/1bBglCIHeIwR78kJXJzkZo',
 'duration_ms': 224554,
 'time_signature': 4}

Schritt 3: Abruf von Song Empfehlungen

In [42]:
RECOMMEND_ENDPOINT = 'https://api.spotify.com/v1/recommendations'

PARAMS = {'seed_tracks': '1bBglCIHeIwR78kJXJzkZo,30dDeLflXjGSv7qjrjmYCG,6gOR8xTSQWEMfEJa7r52k7',
          'limit': 5}

response = requests.get(RECOMMEND_ENDPOINT, headers=HEADER, params=PARAMS)

In [43]:
response

<Response [200]>

In [44]:
response.json()

{'tracks': [{'album': {'album_type': 'SINGLE',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1KpCi9BOfviCVhmpI4G2sY'},
      'href': 'https://api.spotify.com/v1/artists/1KpCi9BOfviCVhmpI4G2sY',
      'id': '1KpCi9BOfviCVhmpI4G2sY',
      'name': 'Tchami',
      'type': 'artist',
      'uri': 'spotify:artist:1KpCi9BOfviCVhmpI4G2sY'},
     {'external_urls': {'spotify': 'https://open.spotify.com/artist/1YYJxpOXYk1z1WtqdeLMkn'},
      'href': 'https://api.spotify.com/v1/artists/1YYJxpOXYk1z1WtqdeLMkn',
      'id': '1YYJxpOXYk1z1WtqdeLMkn',
      'name': 'Habstrakt',
      'type': 'artist',
      'uri': 'spotify:artist:1YYJxpOXYk1z1WtqdeLMkn'},
     {'external_urls': {'spotify': 'https://open.spotify.com/artist/5R5qNSYFn04J9jF8UnmYbK'},
      'href': 'https://api.spotify.com/v1/artists/5R5qNSYFn04J9jF8UnmYbK',
      'id': '5R5qNSYFn04J9jF8UnmYbK',
      'name': 'Honey & Badger',
      'type': 'artist',
      'uri': 'spotify:artist:5R5qNSYFn04J9jF8UnmYbK'}],


### Anwendungsbeispiel 4: Amtliche Statistiken mit Datenguide

Read the [docs](https://datengui.de/docs)

*Hinweis: Da Datenguide open data bereitstellt brauchen wir keine `auth_token`! Der `data` Code `52411:ISV001` kennzeichnet den Datensatz "Statistik über beantragte Insolvenzverfahren - Insolvenzverfahren", der `region` Code `05` steht für "Nordrhein-Westfalen", und es werden alle verfügbaren Daten ab `2008` abgerufen.*

In [45]:
ENDPOINT_DATENGUIDE = 'https://tabular.genesapi.org'

PARAMS = {'data': '52411:ISV001',
          'region': '05',
          'time': '2008:',
          'format':'json'}

response = requests.get(ENDPOINT_DATENGUIDE, params=PARAMS)

In [46]:
response

<Response [200]>

In [47]:
response.json()

{'schema': {'fields': [{'name': 'index', 'type': 'integer'},
   {'name': 'region_id', 'type': 'string'},
   {'name': 'region_name', 'type': 'string'},
   {'name': 'year', 'type': 'string'},
   {'name': 'measure', 'type': 'string'},
   {'name': 'value', 'type': 'integer'},
   {'name': 'statistic', 'type': 'string'}],
  'primaryKey': ['index'],
  'pandas_version': '0.20.0'},
 'data': [{'index': 0,
   'region_id': '05',
   'region_name': 'Nordrhein-Westfalen',
   'year': '2008',
   'measure': 'ISV001',
   'value': 35453,
   'statistic': '52411'},
  {'index': 21,
   'region_id': '05',
   'region_name': 'Nordrhein-Westfalen',
   'year': '2008',
   'measure': 'ISV001',
   'value': 35453,
   'statistic': '52411'},
  {'index': 1,
   'region_id': '05',
   'region_name': 'Nordrhein-Westfalen',
   'year': '2009',
   'measure': 'ISV001',
   'value': 37501,
   'statistic': '52411'},
  {'index': 22,
   'region_id': '05',
   'region_name': 'Nordrhein-Westfalen',
   'year': '2009',
   'measure': 'ISV0

In [50]:
pd.DataFrame(response.json()['data'])

,index,region_id,region_name,year,measure,value,statistic
0,0,05,Nordrhein-Westfalen,2008,ISV001,35453,52411
1,21,05,Nordrhein-Westfalen,2008,ISV001,35453,52411
2,1,05,Nordrhein-Westfalen,2009,ISV001,37501,52411
3,22,05,Nordrhein-Westfalen,2009,ISV001,37501,52411
4,2,05,Nordrhein-Westfalen,2010,ISV001,39786,52411
5,10,05,Nordrhein-Westfalen,2010,ISV001,39786,52411
6,3,05,Nordrhein-Westfalen,2011,ISV001,38625,52411
7,15,05,Nordrhein-Westfalen,2011,ISV001,38625,52411
8,4,05,Nordrhein-Westfalen,2012,ISV001,38116,52411
9,16,05,Nordrhein-Westfalen,2012,ISV001,38116,52411
